In [11]:
import os
print(os.listdir())

['Lab4.ipynb', '.ipynb_checkpoints', 'UNITENReview.csv', 'Review.csv']


In [13]:
# Load dataset
import pandas as pd
file_path = "Review.csv"
df = pd.read_csv(file_path, encoding="latin1")
# Display column content without truncation
pd.set_option('display.max_colwidth', None) # Set to None for unlimited width
print(df)

                                                                           Review
0   The product arrived on time. Packaging was great, and the quality is amazing!
1                                        THIS PRODUCT IS JUST AMAZING! I LOVE IT.
2     I bought this phone for $799, and it has a 120Hz display. Totally worth it!
3                         Wow!!! This product is awesome... but a bit expensive??
4                                             The laptop works perfectly fine.   
5    Check out the full product details here: https://example.com/product-details
6         <div><h2>Great Purchase!</h2><p>I am happy with this product.</p></div>
7                The battry life is excelent, but the chargin cable is too short.
8                       I can't believe it's so good! Didn't expect such quality.
9                   Love this product! ???? Fast delivery ??, amazing quality! ??
10                       TBH, I wasnt expecting much, but OMG, this is awesome!!
11              

In [35]:
# Lowercase conversion
def convert_to_lowercase(text):
 return text.lower()
df["lowercased"] = df["Review"].apply(convert_to_lowercase)
# Display column content without truncation
pd.set_option('display.max_colwidth', None) # Set to None for unlimited width
print(df["lowercased"])

# Removal of URLs
import re
# remove any URLs that start with "http" or "www" from the text
def remove_urls(text):
 return re.sub(r'http\S+|www\S+', '', text)
df["urls_removed"] = df["lowercased"].apply(remove_urls)
# Display column content without truncation
pd.set_option('display.max_colwidth', None) # Set to None for unlimited width
print(df["urls_removed"])

!pip install emoji
# Removal of HTML tags
from bs4 import BeautifulSoup
# extracts only the text, removing all HTML tags
def remove_html_tags(text):
 return BeautifulSoup(text, "html.parser").get_text()
df["html_removed"] = df["urls_removed"].apply(remove_html_tags)
# Display column content without truncation
pd.set_option('display.max_colwidth', None) # Set to None for unlimited width
print(df["html_removed"])

# Removal of emojis (if any)
import emoji
# replace emoji with ''
def remove_emojis(text):
 return emoji.replace_emoji(text, replace='')
df["emojis_removed"] = df["html_removed"].apply(remove_emojis)
# Display column content without truncation
pd.set_option('display.max_colwidth', None) # Set to None for unlimited width
print(df["emojis_removed"])


# Replace Contractions
contractions_dict = {
    "wasn't": "was not", "isn't": "is not", "aren't": "are not", "weren't": "were not",
    "doesn't": "does not", "don't": "do not", "didn't": "did not", "can't": "cannot",
    "couldn't": "could not", "shouldn't": "should not", "wouldn't": "would not",
    "won't": "will not", "haven't": "have not", "hasn't": "has not", "hadn't": "had not",
    "i'm": "i am", "you're": "you are", "he's": "he is", "she's": "she is",
    "it's": "it is", "we're": "we are", "they're": "they are", "i've": "i have",
    "you've": "you have", "we've": "we have", "they've": "they have", "i'd": "i would",
    "you'd": "you would", "he'd": "he would", "she'd": "she would", "we'd": "we would",
    "they'd": "they would", "i'll": "i will", "you'll": "you will", # Fixed syntax error here
    "he'll": "he will", "she'll": "she will", "we'll": "we will", "they'll": "they will",
    "let's": "let us", "that's": "that is", "who's": "who is", "what's": "what is",
    "where's": "where is", "when's": "when is", "why's": "why is"
}

# Build the regex pattern for contractions
escaped_contractions = [re.escape(contraction) for contraction in contractions_dict.keys()]
joined_contractions = "|".join(escaped_contractions)
contractions_pattern = r'\b(' + joined_contractions + r')\b'
compiled_pattern = re.compile(contractions_pattern, flags=re.IGNORECASE)

# Define a function to replace contractions
def replace_contractions(text):
    if pd.isna(text): return ""
    
    # Normalize curly apostrophes (smart quotes) to match dict keys
    text = str(text).replace("’", "'")
    
    def replace_match(match):
        matched_word = match.group(0).lower() 
        return contractions_dict[matched_word]
    
    return compiled_pattern.sub(replace_match, text)

# Apply to the column (using 'slangs_replaced' as per your original code)
df["contractions_replaced"] = df["slangs_replaced"].apply(replace_contractions)

# --- Display Results ---
pd.set_option('display.max_colwidth', None)
print(df["contractions_replaced"])

# Remove punctuations and special characters
import string
# Function to remove punctuation
def remove_punctuation(text):
 return text.translate(str.maketrans('', '', string.punctuation))
# Apply the function to the column
df["punctuations_removed"] = df["contractions_replaced"].apply(remove_punctuation)
# Display column content without truncation
pd.set_option('display.max_colwidth', None) # Set to None for unlimited width
print(df["punctuations_removed"])

# Remove numbers
def remove_numbers(text):
 return re.sub(r'\d+', '', text) # Removes all numeric characters
# Apply the function to the column
df["numbers_removed"] = df["punctuations_removed"].apply(remove_numbers)
# Display column content without truncation
pd.set_option('display.max_colwidth', None) # Set to None for unlimited width
print(df["numbers_removed"])

# Correct spelling mistakes
from autocorrect import Speller
# Initialize spell checker
spell = Speller(lang='en')
# Function to correct spelling
def correct_spelling(text):
 return spell(text) # Apply correction
# Apply the function to the column
df["spelling_corrected"] = df["numbers_removed"].apply(correct_spelling)

# Display column content without truncation
pd.set_option('display.max_colwidth', None) # Set to None for unlimited width
print(df["spelling_corrected"])

# Remove stopwords
import nltk
from nltk.corpus import stopwords
import pandas as pd

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Define stopwords list
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()  # Split text into words
    filtered_words = []   # Create an empty list
    
    for word in words:  # Loop through each word
        lower_word = word.lower()  # Convert to lowercase
        
        if lower_word not in stop_words:  # Check if NOT a stopword
            filtered_words.append(word)   # Add to filtered list
            
    return " ".join(filtered_words)  # Join back into sentence

# Apply the function to the column
df["stopwords_removed"] = df["spelling_corrected"].apply(remove_stopwords)

# Display column content without truncation
pd.set_option('display.max_colwidth', None)
print(df["stopwords_removed"])

# Stemming - reduces words to their base root by chopping off suffixes
from nltk.stem import PorterStemmer
import pandas as pd

# Initialize the stemmer
stemmer = PorterStemmer()

# Function to apply stemming
def stem_text(text):
    if not isinstance(text, str):
        return ""

    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]  # Apply stemming
    return " ".join(stemmed_words)

# Apply the function
df["stemmed_words"] = df["stopwords_removed"].apply(stem_text)

# Display column content without truncation
pd.set_option('display.max_colwidth', None)
print(df["stemmed_words"])

import nltk
# Download the required resources
nltk.download('wordnet') # For lemmatization
nltk.download('omw-1.4') # WordNet lexical database
nltk.download('averaged_perceptron_tagger_eng') # For POS tagging
nltk.download('punkt_tab') # For tokenization
# Lemmatization - reduces words to their base dictionary form (lemma)
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to map NLTK POS tags to WordNet POS tags
def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):  # Adjective
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):  # Verb
        return wordnet.VERB
    elif nltk_tag.startswith('N'):  # Noun
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):  # Adverb
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

# Function to lemmatize text with POS tagging
def lemmatize_text(text):
    if not isinstance(text, str):
        return ""

    words = word_tokenize(text)
    pos_tags = pos_tag(words)

    # Lemmatize each word with its correct POS tag
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in pos_tags
    ]

    return " ".join(lemmatized_words)

# Apply the function
df["lemmatized"] = df["stopwords_removed"].apply(lemmatize_text)

# Display column content without truncation
pd.set_option('display.max_colwidth', None)
print(df["lemmatized"])

df.to_csv("Processed_Reviews.csv", index=False)

0     the product arrived on time. packaging was great, and the quality is amazing!
1                                          this product is just amazing! i love it.
2       i bought this phone for $799, and it has a 120hz display. totally worth it!
3                           wow!!! this product is awesome... but a bit expensive??
4                                               the laptop works perfectly fine.   
5      check out the full product details here: https://example.com/product-details
6           <div><h2>great purchase!</h2><p>i am happy with this product.</p></div>
7                  the battry life is excelent, but the chargin cable is too short.
8                         i can't believe it's so good! didn't expect such quality.
9                     love this product! ???? fast delivery ??, amazing quality! ??
10                         tbh, i wasnt expecting much, but omg, this is awesome!!
11                            this is the best product i have ever used in m

ModuleNotFoundError: No module named 'emoji'

In [34]:
df.to_csv("Processed_Reviews.csv", index=False)